# Main Analysis File
This file contains the main data analysis code for the project. It is divided into 3 main parts:
1. Data Cleaning
2. Exploratory Data Analysis
3. Machine Learning

In [1]:
# CORE
import pandas as pd
import os
import numpy as np  # Numpy for numerical computations and array operations
import pandas as pd  # Pandas for data manipulation and analysis

# MACHINE LEARNING & STATISTICS 
import scipy.stats as stats  # SciPy for scientific computing and technical computing, including statistics
import sklearn as sk # Scikit-learn for machine learning and predictive modeling

# VISUALIZATION
import matplotlib.pyplot as plt  # Matplotlib for creating static, animated, and interactive visualizations
import seaborn as sns  # Seaborn for statistical data visualization built on top of Matplotlib
import plotly.express as px  # Plotly Express for creating interactive plots and charts

/Users/jerhadf/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Cleaning

In [4]:
# filepaths of all of the csv data files to be analyzed

# autonest_csv = "CEIP_csv/AutoNest.csv"
# autonest_strategy_csv = "CEIP_csv/AutoNestStrategy.csv"
# material_csv = "CEIP_csv/Material.csv"
# nest_csv = "CEIP_csv/Nest.csv"
# part_csv = "CEIP_csv/Part.csv"
# performance_csv = "CEIP_csv/Performance.csv"
training_csv = "../CEIP_csv/training_data.csv"

# read in all of these csv files as pandas dataframes

# autonest_df = pd.read_csv(autonest_csv)
# autonest_strategy_df = pd.read_csv(autonest_strategy_csv)
# material_df = pd.read_csv(material_csv)
# nest_df = pd.read_csv(nest_csv)
# part_df = pd.read_csv(part_csv)
# performance_df = pd.read_csv(performance_csv)
training_df = pd.read_csv(training_csv) # takes about 1 min to read 

#### Data Previewing

In [8]:
# count the number of null values 
# training_df.isnull().sum() # --> RESULT: no null values in any column  

column_list = list(training_df.columns)
print(column_list)

# count the values for the ixMaterial column in the training_df 
# training_df.ixMaterial.value_counts()

# plot the distribution of the ixMaterial column in the training_df
# only plot the 10 most common values
# training_df.ixMaterial.value_counts().nlargest(10).plot(kind='bar', figsize=(10,5))

['ixJobSummary', 'ixNest', 'ixPart', 'dPartTrueArea', 'cRequired', 'cNested', 'ixMaterial', 'fExtShape', 'dExtArea', 'dExtBoundaryDist', 'dExtContainedDist', 'dLgIntArea', 'dLgIntBoundaryDist', 'dLgIntContainedDist', 'dLgExtConArea', 'dLgExtConBoundaryDist', 'dLgExtConContainedDist', 'cTimesCut', 'dNestingTime', 'fStrategies', 'dSheetLength', 'dSheetWidth', 'dSheetArea', 'dLengthUsed', 'dWidthUsed', 'dPartArea', 'calcUtil', 'ixAutoNestStrategy', 'fAllPartsNested']


In [10]:
# count the number of unique values
training_df.nunique(axis=0)

ixJobSummary               224892
ixNest                     224892
ixPart                    4200357
dPartTrueArea              984974
cRequired                    1534
cNested                      1654
ixMaterial                   7316
fExtShape                      52
dExtArea                   825580
dExtBoundaryDist           159907
dExtContainedDist          526206
dLgIntArea                  74924
dLgIntBoundaryDist          35721
dLgIntContainedDist         83215
dLgExtConArea               71156
dLgExtConBoundaryDist       24334
dLgExtConContainedDist      68328
cTimesCut                     176
dNestingTime                23997
fStrategies                   292
dSheetLength                 2917
dSheetWidth                  1937
dSheetArea                  13217
dLengthUsed                136950
dWidthUsed                 112688
dPartArea                  168268
calcUtil                   172012
ixAutoNestStrategy             13
fAllPartsNested                 2
dtype: int64

In [11]:
# above shows that there are only 4.2 million unique values for ixPart 
# this indicates that there are a lot of rows that are duplicated? 

# remove duplicates 
training_df = training_df.drop_duplicates()
training_df.shape # --> (5,762,622 rows, 29 columns)

(5762622, 29)

In [ ]:
# Group by one or more columns and calculate the mean of a specific column
# grouped_data = df.groupby(['column1', 'column2']).mean()
# Perform a cross-tabulation between two columns 
# cross_tab = pd.crosstab(df['column1'], df['column2'])

#### Re-encoding Variables
* One-hot encode all variables that are numerical but represent categories
* Replace ixMaterial and ixAutoNestStrategy with their appropriate values from the JSON file 
* Limit to only Materials that are steel

In [ ]:
# replace ixMaterial column with the values from the MaterialTypes.json file 
# read in the json file as a pandas dataframe
material_types_df = pd.read_json('../CEIP_csv/MaterialTypes.json')

# create a dictionary of the material types and their corresponding values 
material_types_dict = material_types_df.to_dict()['ixMaterial']

# replace the values in the ixMaterial column with the corresponding material type
training_df.ixMaterial = training_df.ixMaterial.replace(material_types_dict)

# df_cleaned['column'] = df_cleaned['column'].replace({'old_value': 'new_value'})


# # Apply custom functions to clean data (e.g., using .apply() or .map())
# def clean_function(value):
#     # Your cleaning logic here
#     return cleaned_value

In [ ]:
# one hot encoding for the ixMaterial column
training_df = pd.get_dummies(training_df, columns=['ixMaterial'])

# One-hot encoding
# df_cleaned = pd.get_dummies(df_cleaned, columns=['categorical_column'])

# # Label encoding
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# df_cleaned['categorical_column'] = le.fit_transform(df_cleaned['categorical_column'])

# # Ordinal encoding
# from sklearn.preprocessing import OrdinalEncoder
# encoder = OrdinalEncoder()
# df['ordinal_encoded_feature'] = encoder.fit_transform(df[['categorical_column']]) 

# # Target encoding
# from category_encoders import TargetEncoder
# te = TargetEncoder()
# df_cleaned['categorical_column'] = te.fit_transform(df_cleaned['categorical_column'], df_cleaned['target_column'])

In [ ]:
# filter to only materials that are in the SteelMaterials.json file 
# read in the json file as a pandas dataframe
steel_materials_df = pd.read_json('../CEIP_csv/SteelMaterials.json')

# create a list of the steel materials 
steel_materials_list = list(steel_materials_df.ixMaterial)

# filter the training_df to only include steel materials
training_df = training_df[training_df.ixMaterial.isin(steel_materials_list)]

#### Other data cleaning

In [ ]:
# Convert a numeric variable to categorical using custom ranges
# bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
# df['age_group'] = pd.cut(df['age'], bins=bins, labels=lab

#### Downsampling Data
* Downsampling data to ~1,000,000 samples for early machine learning & analysis

In [ ]:
# downsampling code 

### Removing outliers

In [ ]:
# Visualize outliers using a boxplot
# sns.boxplot(data=df_cleaned['column'])
# plt.show()

In [ ]:
# # Calculate the IQR for a specific column
# Q1 = df_cleaned['column'].quantile(0.25)
# Q3 = df_cleaned['column'].quantile(0.75)
# IQR = Q3 - Q1

# # Remove outliers based on the IQR
# df_cleaned = df_cleaned[(df_cleaned['column'] >= Q1 - 1.5 * IQR) & (df_cleaned['column'] <= Q3 + 1.5 * IQR)]

# # Alternative: Remove rows with values outside a specified range
# df = df[(df['variable'] >= lower_bound) & (df['variable'] <= upper_bound)]

In [ ]:
# Convert the column to numeric data type (if not already numeric)
# df[var] = pd.to_numeric(df[var], errors='coerce')

# # Calculate the IQR and define the threshold values
# Q1 = df[var].quantile(0.25)
# Q3 = df[var].quantile(0.75)
# IQR = Q3 - Q1
# threshold_low = Q1 - 1.5 * IQR
# threshold_high = Q3 + 1.5 * IQR

# Create a new DataFrame with the outlier values removed
# df_no_outliers = df[(df[var] >= threshold_low) & (df[var] <= threshold_high)]

# Define the outlier detection function using the IQR method
# def remove_outliers_iqr(df, column):
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1
#     threshold_low = Q1 - 1.5 * IQR
#     threshold_high = Q3 + 1.5 * IQR
#     return df[(df[column] >= threshold_low) & (df[column] <= threshold_high)]

# # Remove outliers from all columns in the DataFrame
# for column in df.columns:
#     df = remove_outliers_iqr(df, column)


#### Feature creation
* Making new features based on Mark's - what can we make? See the CEIP_db/Mark_calculations folder for more details on what he did 

In [ ]:
# # Create a new feature by combining two existing features
# df['new_feature'] = df['column1'] * df['column2']

# # Create a new feature by applying a mathematical operation
# df['log_feature'] = np.log(df['column1'])

# # Create a new feature by applying a custom function
# def custom_function(x):
#     return x**2

# df['squared_feature'] = df['column1'].apply(custom_function)

#### Scaling & Normalization
Bring features to a similar scale to prevent one from dominating the  - some ML models sensitive to feature magnitudes, can perform poorly on different scales

In [ ]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

# # Standard Scaling (Z-score normalization)
# scaler = StandardScaler()
# df['standard_scaled_feature'] = scaler.fit_transform(df[['column1']])

# # Min-Max Scaling (Normalization)
# scaler = MinMaxScaler()
# df['min_max_scaled_feature'] = scaler.fit_transform(df[['column1']])

# Robust Scaling: Scale features using median and interquartile range, making it less sensitive to outliers

## Exploratory Data Analysis

In [ ]:
# automated EDA with AutoViz
eda()

### Exploratory Data Visualizations

In [ ]:
# histograms
# plt.hist(df_cleaned['column'], bins=20)
# plt.xlabel('Column Name')
# plt.ylabel('Frequency')
# plt.title('Histogram of Column Name')
# plt.show()

# box plots 
# sns.boxplot(x=df_cleaned['column'])
# plt.xlabel('Column Name')
# plt.title('Box Plot of Column Name')
# plt.show()

In [ ]:
# scatter plots 
# plt.scatter(df_cleaned['column1'], df_cleaned['column2'])
# plt.xlabel('Column 1')
# plt.ylabel('Column 2')
# plt.title('Scatter Plot of Column 1 vs. Column 2')
# plt.show()

# interactive scatter plot 
# fig = px.scatter(df_cleaned, x='column1', y='column2', color='category_column', hover_data=['column3'])
# fig.show()

In [ ]:
# bubble plots - 3 numerical variables
# plt.scatter(df_cleaned['column1'], df_cleaned['column2'], s=df_cleaned['column3'], alpha=0.5)
# plt.xlabel('Column 1')
# plt.ylabel('Column 2')
# plt.title('Bubble Plot of Column 1 vs. Column 2 (Size = Column 3)')
# plt.show()

In [ ]:
# parallel coordinates plot for multiple numerical variables
# from pandas.plotting import parallel_coordinates

# parallel_coordinates(df_cleaned[['column1', 'column2', 'column3', 'category_column']], 'category_column')
# plt.title('Parallel Coordinates Plot')
# plt.show()

In [ ]:
# regression plotting 
# sns.regplot(x='column1', y='column2', data=df_cleaned)
# plt.title('Regression Plot')
# plt.show()

In [ ]:
# Things to visualize: 
# tradeoffs between nesting time and utilization
# distribution of utilization 
# relationship between fStrategies and utilization
# visualization of the frequency of different Strategies used
# visualization of the frequency of different Materials used 
# distribution of Parts, number of Parts per job, number of Parts nested 


### Correlation analysis

In [ ]:
# Select the variable of interest
# selected_var = 'dCropUtil'

# # Compute the correlation coefficients between the selected variable and all other variables
# corr_values = df.corr()[selected_var]

# # Compute the correlation coefficients between all pairs of variables
# corr_values = df.corr()

# # Display the correlation coefficients
# print(corr_values)

In [ ]:
# # Display the correlation matrix as a heatmap for better visualization
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
# plt.show()

In [ ]:
# Plot a histogram of the selected variable
# sns.histplot(data=df_no_outliers, x=selected_var, kde=True)
# plt.title('Histogram of {}'.format(selected_var))
# plt.xlabel(selected_var)
# plt.ylabel('Frequency')
# plt.show()

# # Plot a boxplot of the selected variable
# sns.boxplot(data=df_no_outliers, x=selected_var)
# plt.title('Boxplot of {}'.format(selected_var))
# plt.xlabel(selected_var)
# plt.show()

# # Compute the summary statistics of the selected variable
# summary = df_no_outliers[selected_var].describe()
# print(summary)

In [ ]:
# remove highly correlated features
# corr_matrix = df_cleaned.corr().abs()
# upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# cols_to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > correlation_threshold)]
# df_cleaned.drop(cols_to_drop, axis=1, inplace=True)

### Clustering

In [ ]:
# from sklearn.cluster import KMeans

# # Create a K-means clustering model with 3 clusters
# kmeans = KMeans(n_clusters=3, random_state=0).fit(df)

# # Get the cluster labels for each data point
# labels = kmeans.labels_

# # Add cluster labels to the original DataFrame
# df['Cluster'] = labels

In [ ]:
# with PyCaret
# import pycaret clustering and init setup
# see https://colab.research.google.com/github/pycaret/pycaret/blob/master/tutorials/Tutorial%20-%20Clustering.ipynb#scrollTo=4181de41
from pycaret.clustering import *
s = setup(data, session_id = 123)

# import ClusteringExperiment and init the class
from pycaret.clustering import ClusteringExperiment
exp = ClusteringExperiment()

# init setup on exp
exp.setup(data, session_id = 123)

# train kmeans model
kmeans = create_model('kmeans')

# to check all the available models
models()

# train meanshift model
meanshift = create_model('meanshift')

kmeans_cluster = assign_model(kmeans)
kmeans_cluster

# plot pca cluster plot 
plot_model(kmeans, plot = 'cluster')

# plot elbow
plot_model(kmeans, plot = 'elbow')

# plot silhouette
plot_model(kmeans, plot = 'silhouette')

evaluate_model(kmeans)

# predict on test set
kmeans_pred = predict_model(kmeans, data=data)
kmeans_pred

## Machine Learning

### Model exploration with PyCaret

Try PyCaret to start 
https://github.com/pycaret/pycaret/blob/master/tutorials/Tutorial%20-%20Regression.ipynb

In [ ]:
data = training_df

# import pycaret regression and init setup
from pycaret.regression import *
s = setup(data, target = 'charges', session_id = 123)

# import RegressionExperiment and init the class
from pycaret.regression import RegressionExperiment
exp = RegressionExperiment()

# init setup on exp
exp.setup(data, target = 'charges', session_id = 123)

# compare baseline models
best = compare_models()

# compare models using OOP
# exp.compare_models()

In [ ]:
# evaluate the models and interpret 
# plot residuals
plot_model(best, plot = 'residuals')

# plot error
plot_model(best, plot = 'error')

# plot feature importance
plot_model(best, plot = 'feature')

evaluate_model(best)

# train lightgbm model
lightgbm = create_model('lightgbm')

# interpret summary model
interpret_model(lightgbm, plot = 'summary')

In [ ]:
# predict on test set
holdout_pred = predict_model(best)

# show predictions df
holdout_pred.head()

# copy data and drop charges
new_data = data.copy()
new_data.drop('charges', axis=1, inplace=True)
new_data.head()

# predict model on new_data
predictions = predict_model(best, data = new_data)
predictions.head()


# save pipeline
save_model(best, 'my_first_pipeline')

# load pipeline
loaded_best_pipeline = load_model('my_first_pipeline')
loaded_best_pipeline

In [ ]:
# ensembling 

# train a dt model with default params
dt = create_model('dt')

# tune hyperparameters of dt
tuned_dt = tune_model(dt)

# define tuning grid
dt_grid = {'max_depth' : [None, 2, 4, 6, 8, 10, 12]}

# tune model with custom grid and metric = MAE
tuned_dt = tune_model(dt, custom_grid = dt_grid, optimize = 'MAE')

# tune dt using optuna
tuned_dt = tune_model(dt, search_library = 'optuna')

# ensemble with bagging
ensemble_model(dt, method = 'Bagging')

# ensemble with boosting
ensemble_model(dt, method = 'Boosting')

In [ ]:
# # top 3 models based on mae
best_mae_models_top3 = compare_models(n_select = 3, sort = 'MAE')

# blending models 
blend_models(best_mae_models_top3)

In [ ]:
# find best model based on CV metrics
# returns the best model out of all trained models in the current setup based on the optimize parameter
automl() 

In [ ]:
# dashboard function
dashboard(dt, display_format ='inline')

In [ ]:
# finalize the model 
final_best = finalize_model(best)

# save model
# save_model(best, 'my_first_model')


# load model
# loaded_from_disk = load_model('my_first_model')
# loaded_from_disk



### Basic models to try 
* Linear Regression - Use sci-kit learn to implement
* Logistic Regression
* Support Vector Machines
* Basic decision trees
* Naive Bayes

### XGBoost 
Tree ensemble model that can handle tabular, numerical, low-dimensional data very well. Fast and scalable, and can be hyperparameter tuned to optimize performance.

### ResNet-like architecture
Neural network architecture adapted for tabular data, allowing learning from shallow and deep features. Use PyTorch or TensorFlow to implement. Less interpretable. 

### Transformer 
Use Transformers with tabular data? 

### KNN - K Nearest Neighbors
Predicts the target variable based on the similarity of the features with the nearest neighbors in the training dataset. Computationally expensive, but can be used for regression and classification.

### Model Evaluation
* Cross-validation
* Classification Accuracy
* Confusion Matrix
* ROC-AUC Curve
  

### Model Fine-tuning
* Hyperparameter tuning
* Ensembling multiple models together - voting classifier, bagging, boosting, XGBoost 

### Feature Importance

**Univariate Feature Selection:** This method uses statistical tests like chi-squared tests or ANOVA to evaluate the relationship between each feature and the target variable independently. It ranks the features based on their significance.

In [13]:
# from sklearn.feature_selection import SelectKBest, chi2, f_classif

# # For categorical target using Chi-squared test
# selector = SelectKBest(score_func=chi2, k=5)
# selector.fit_transform(X, y)

# # For continuous target using ANOVA F-value
# selector = SelectKBest(score_func=f_classif, k=5)
# selector.fit_transform(X, y)

# # Get feature importances
# scores = selector.scores_
# feature_importances = pd.DataFrame({'feature': X.columns, 'importance': scores})
# feature_importances = feature_importances.sort_values('importance', ascending=False)
# print(feature_importances)

In [ ]:
# other statistical testing 
# import scipy.stats as stats

# # Perform a t-test to compare the means of two groups
# t_stat, p_value = stats.ttest_ind(group1, group2)
# print("T-statistic:", t_stat, "P-value:", p_value)

# # Perform a chi-squared test to determine the association between two categorical variables
# chi_stat, p_value, dof, ex = stats.chi2_contingency(contingency_table)
# print("Chi-squared statistic:", chi_stat, "P-value:", p_value)